# Acceso a Pfam

Buscamos datos de todas la familias de Pfam

La url base de Pfam es "https://pfam.xfam.org/", pero en algunos sistemas no se
puede acceder bien, en esos casos, se puede usar "http://pfam.xfam.org/" (notar que es "https" en un
caso y "http" en el otro.

Pfam provee unos pocos servicios que se pueden usar para acceder programaticamente.

In [27]:
library(httr)
library(xml2)

## Recuperar datos de las familias disponibles

Uno de los servicios permite recuperar el nombre, código de acceso y descripción de todas las familias de proteínas disponibles.

In [6]:
url   <- "http://pfam.xfam.org/families"
respuesta <- GET(url, query=list(output="text"))

In [12]:
families <-read.table(
    text=content(respuesta),
    header=FALSE,
    sep="\t",
    quote="",
    comment.char="#",
    col.names=c('Accession', 'Name', 'Description')
)
head(families)

,Accession,Name,Description
,<chr>,<chr>,<chr>
1,PF00001,7tm_1,7 transmembrane receptor (rhodopsin family)
2,PF00002,7tm_2,7 transmembrane receptor (Secretin family)
3,PF00003,7tm_3,7 transmembrane sweet-taste receptor of 3 GCPR
4,PF00004,AAA,ATPase family associated with various cellular activities (AAA)
5,PF00005,ABC_tran,ABC transporter
6,PF00006,ATP-synt_ab,"ATP synthase alpha/beta family, nucleotide-binding domain"


## Conversión de identificadores

Podemos convertit el código de acceso y el identificador de nombres.


In [51]:
# url <- "http://pfam.xfam.org/family/id?output=xml&acc=PF00082"
url <- "http://pfam.xfam.org/family/id"

family_acc <- 'PF00082'
respuesta <- GET(url, query=list(output="xml", acc=family_acc))
data <- content(respuesta)

In [52]:
xml_attrs(xml_children(data))

[[1]]
    entry_type      accession             id 
      "Pfam-A"      "PF00082" "Peptidase_S8"

## Recuperar anotaciones

In [58]:
url <- "http://pfam.xfam.org/family"

family_acc <- 'PF00082'
respuesta <- GET(url, query=list(output="xml", acc=family_acc))
data <- as_list(content(respuesta))

In [81]:
names(data$pfam$entry)

[1] "description"      "comment"          "go_terms"         "curation_details"
[5] "hmm_details"

In [82]:
names(data$pfam$entry$curation_details)

[1] "status"              "seed_source"         "previous_id"        
 [4] "num_archs"           "num_seqs"            "num_species"        
 [7] "num_structures"      "percentage_identity" "av_length"          
[10] "av_coverage"         "type"

Recuperamos el tipo de familia, la descripción, la cobertura promedio, y la longitud promedio.

In [80]:
# descripcion
trimws(data$pfam$entry$description)

# tipo
unlist(data$pfam$entry$curation_details$type)

# cobertura promedio
unlist(data$pfam$entry$curation_details$av_coverage)

# cobertura promedio
unlist(data$pfam$entry$curation_details$av_length)

[1] "Subtilase family"

[1] "Domain"

[1] "44.46"

[1] "302.00"

## Buscar datos de una proteína

Podemos buscar si una proteína 

In [154]:
url <- "http://pfam.xfam.org/protein"

uniprot <- 'P0AA25'
respuesta <- GET(url, query=list(output="xml", entry=uniprot))
data <- content(respuesta)
data_list <- as_list(data)

In [155]:
data_list

$pfam
$pfam$entry
$pfam$entry$description
$pfam$entry$description[[1]]
[1] "\nThioredoxin 1\n"


$pfam$entry$taxonomy
$pfam$entry$taxonomy[[1]]
[1] "Bacteria; Proteobacteria; Gammaproteobacteria; Enterobacterales; Enterobacteriaceae; Escherichia."

attr(,"tax_id")
[1] "83333"
attr(,"species_name")
[1] "Escherichia coli (strain K12)"

$pfam$entry$sequence
$pfam$entry$sequence[[1]]
[1] "MSDKIIHLTDDSFDTDVLKADGAILVDFWAEWCGPCKMIAPILDEIADEYQGKLTVAKLNIDQNPGTAPKYGIRGIPTLLLFKNGEVAATKVGALSKGQLKEFLDANLA"

attr(,"length")
[1] "109"
attr(,"md5")
[1] "850143d49315e8891aad60bd107000ef"
attr(,"crc64")
[1] "EF5933EA29668EE9"
attr(,"version")
[1] "2"

$pfam$entry$matches
$pfam$entry$matches$match
$pfam$entry$matches$match$location
list()
attr(,"start")
[1] "4"
attr(,"end")
[1] "106"
attr(,"ali_start")
[1] "5"
attr(,"ali_end")
[1] "105"
attr(,"hmm_start")
[1] "2"
attr(,"hmm_end")
[1] "102"
attr(,"evalue")
[1] "1.1e-30"
attr(,"bitscore")
[1] "116.90"

attr(,"accession")
[1] "PF00085"
attr(,"id")
[1] "Thioredoxin"
attr(,"type")
[1] "Pfam-A"


attr(,"entry_type")
[1] "sequence"
attr(,"db")
[1] "uniprot"
attr(,"db_release")
[1] "2020_06"
attr(,"accession")
[1] "P0AA25"
attr(,"id")
[1] "THIO_ECOLI"

attr(,"schemaLocation")
[1] "https://pfam.xfam.org/                           https://pfam.xfam.org/static/documents/schemas/protein.xsd"
attr(,"release")
[1] "34.0"
attr(,"release_date")
[1] "2021-03-04"
attr(,"xmlns:xsi")
[1] "http://www.w3.org/2001/XMLSchema-instance"
attr(,"xmlns")
[1] "https://pfam.xfam.org/"

In [156]:
# Recupero la secuencia en formato fasta
data_list$pfam$entry$sequence

[[1]]
[1] "MSDKIIHLTDDSFDTDVLKADGAILVDFWAEWCGPCKMIAPILDEIADEYQGKLTVAKLNIDQNPGTAPKYGIRGIPTLLLFKNGEVAATKVGALSKGQLKEFLDANLA"

attr(,"length")
[1] "109"
attr(,"md5")
[1] "850143d49315e8891aad60bd107000ef"
attr(,"crc64")
[1] "EF5933EA29668EE9"
attr(,"version")
[1] "2"

In [152]:
# Recupero datos del match de la proteína los pfams
matches <- xml_find_all(data, ".//d1:match")

# Datos del match
xml_attrs(matches[[1]])

# Datos de la unibación del match
xml_attrs(xml_children(matches[[1]]))



accession            id          type 
    "PF00085" "Thioredoxin"      "Pfam-A"

[[1]]
    start       end ali_start   ali_end hmm_start   hmm_end    evalue  bitscore 
      "4"     "106"       "5"     "105"       "2"     "102" "1.1e-30"  "116.90"

In [160]:
## Buscar dominios en un secuencia de una proteína
url <- "https://pfam.xfam.org/search/sequence"
seq <- "NQEFLSQEFPDIHILKIDVDECEELTVDYEIQDMPTIKKKKTLESFTGANIEQVKHYS"

body <- list(
    seq = seq,
    output="xml"
)

response <- POST(
    url,
    body=body
)

In [167]:
data <- as_list(content(response))
data$html$body$jobs$job$result_url

[[1]]
[1] "/search/sequence/resultset/61D23A1C-CB12-11EB-A1FF-987E6EC84B9E?output=xml"

In [172]:
url <- paste0("https://pfam.xfam.org/", unlist(data$html$body$jobs$job$result_url))
response_job <- GET(url, query=list(output="xml"))

In [195]:
cat(content(response_job, as="text"))

<?xml version="1.0" encoding="UTF-8"?>
<pfam xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
      xmlns="https://pfam.xfam.org/"
      xsi:schemaLocation="https://pfam.xfam.org/
                          https://pfam.xfam.org/static/documents/schemas/pfam_family.xsd"
      release="34.0"
      release_date="2021-03-04">
  <results job_id="61D23A1C-CB12-11EB-A1FF-987E6EC84B9E">
    <matches>
      <protein length="58">
        <database id="pfam" release="34.0" release_date="2021-03-04">
          <match accession="PF00085.20" id="Thioredoxin" type="Pfam-A" class="Domain">
            <location start="2" end="57" ali_start="5" ali_end="51" hmm_start="43" hmm_end="93" evalue="1.0e-06" bitscore="28.7" evidence="hmmer v3.1b2" significant="1">
              <hmm>
                <![CDATA[laeelkdkvklakvdvdenedlakkyevegiPtlklfkngkevkkyegare]]>
              </hmm>
              <match_string>
                <![CDATA[l++e+  ++ ++k+dvde+e+l  +ye++ +Pt+   k+ k+ ++++ga+ ]]>
              

In [222]:
data <- content(response_job)

matches <- xml_find_all(data, './/match')

xml_attrs(matches)

[[1]]
    accession            id          type         class 
 "PF00085.20" "Thioredoxin"      "Pfam-A"      "Domain"

In [225]:
location <- xml_child(matches, "location")
xml_attrs(location)

[[1]]
         start            end      ali_start        ali_end      hmm_start 
           "2"           "57"            "5"           "51"           "43" 
       hmm_end         evalue       bitscore       evidence    significant 
          "93"      "1.0e-06"         "28.7" "hmmer v3.1b2"            "1"

## Descargar alineamientos

Podemos descargas varios tipos de alineamientos de pfam


In [228]:
# Descargamos alineamientos Seed
accession <- "PF00085"
url <- paste0("https://pfam.xfam.org/family/", accession, "/alignment/seed")
seed_msa <- GET(url)

In [231]:
# Lo guardamos en el disco
write(content(seed_msa, as="text"), file="pf_00085_seed.stockholm")

# STOCKHOLM 1.0
#=GF ID   Thioredoxin
#=GF AC   PF00085.22
#=GF DE   Thioredoxin
#=GF PI   thiored; 
#=GF AU   Sonnhammer ELL;0000-0002-9015-5588
#=GF AU   Eddy SR;0000-0001-6676-4706
#=GF SE   Prosite
#=GF GA   23.50 23.50;
#=GF TC   23.50 23.50;
#=GF NC   23.40 23.40;
#=GF BM   hmmbuild HMM.ann SEED.ann
#=GF SM   hmmsearch -Z 57096847 -E 1000 --cpu 4 HMM pfamseq
#=GF TP   Domain
#=GF WK   Thioredoxin
#=GF WK   Thioredoxin_fold
#=GF WK   Thioredoxin_domain
#=GF CL   CL0172
#=GF DR   INTERPRO; IPR013766;
#=GF DR   PROSITE; PDOC00172;
#=GF DR   PRINTS; PR00421;
#=GF DR   SCOP; 3trx; fa;
#=GF DR   HOMSTRAD; thiored;
#=GF DR   SO; 0000417; polypeptide_domain;
#=GF CC   Thioredoxins are small enzymes that participate in redox
#=GF CC   reactions, via the reversible oxidation of an active centre
#=GF CC   disulfide bond. Some members with only the active site are not
#=GF CC   separated from the noise.
#=GF SQ   34
#=GS I3M694_ICTTR/22-125   AC I3M694.2
#=GS Q38AE1_TRYB2/20-121   AC Q38AE1.

In [ ]:
# Descargamos alineamientos full

accession <- "PF00085"
url <- paste0("https://pfam.xfam.org/family/", accession, "/alignment/full")
full_msa <- GET(url)

In [234]:
# Lo guardamos en el disco
write(content(full_msa, as="text"), file="pf_00085_full.stockholm")

Podemos descargar  proteomas representativos con diferente grado de indentidad de secuencia,
que pertenecen a una familia.

- RP15 Proteinas con 15% de identidad o mayor
- RP35 Proteinas con 35% de identidad o mayor
- RP55 Proteinas con 55% de identidad o mayor
- RP75 Proteinas con 75% de identidad o mayor

Para ello, hay que modificar la url.

In [ ]:
url <- paste0("https://pfam.xfam.org/family/", accession, "/alignment/rp15")
url <- paste0("https://pfam.xfam.org/family/", accession, "/alignment/rp35")
url <- paste0("https://pfam.xfam.org/family/", accession, "/alignment/rp55")
url <- paste0("https://pfam.xfam.org/family/", accession, "/alignment/rp75")

# Acceso a RSCB PDB

Al igual que NCBI y Uniprot, PDB permite realizar búsqueda de texto.
Aunque estas son bastante más complicadas de realizar que en los servicios anteriores.

La consulta se tiene que escribir en formato **json**.

Vamos a provar con una búsquea simple.

La respuesta también está en formato json.

In [4]:
library(httr)

In [39]:
url <- "https://search.rcsb.org/rcsbsearch/v1/query"

query <- '{
  "request_options": {
     "return_all_hits": true
  },
  "query": {
    "type": "terminal",
    "service": "text",
    "parameters": {
      "value": "phosphofructokinase"
    }
  },
  "return_type": "entry"
}'

respuesta <- GET(url, query=list(json=query))

In [40]:
pdb_search <- content(respuesta)

In [41]:
# Recuperamos la cantidad de hits.
pdb_search$total_count

[1] 230

In [43]:
# Recuperamos todos los resultados
length(pdb_search$result_set)

[1] 230

In [47]:
pdb_ids <- sapply(pdb_search$result_set, function(x) x$identifier)

In [46]:
install.packages("Rpdb")
library(bio3d)
library(Rpdb)

also installing the dependencies 'ps', 'fs', 'processx', 'sass', 'jquerylib', 'callr', 'httpuv', 'xtable', 'sourcetools', 'commonmark', 'bslib', 'miniUI', 'webshot', 'shinyjs', 'shiny', 'manipulateWidget', 'rgl'




package 'ps' successfully unpacked and MD5 sums checked
package 'fs' successfully unpacked and MD5 sums checked
package 'processx' successfully unpacked and MD5 sums checked
package 'sass' successfully unpacked and MD5 sums checked
package 'jquerylib' successfully unpacked and MD5 sums checked
package 'callr' successfully unpacked and MD5 sums checked
package 'httpuv' successfully unpacked and MD5 sums checked
package 'xtable' successfully unpacked and MD5 sums checked
package 'sourcetools' successfully unpacked and MD5 sums checked
package 'commonmark' successfully unpacked and MD5 sums checked
package 'bslib' successfully unpacked and MD5 sums checked
package 'miniUI' successfully unpacked and MD5 sums checked
package 'webshot' successfully unpacked and MD5 sums checked
package 'shinyjs' successfully unpacked and MD5 sums checked
package 'shiny' successfully unpacked and MD5 sums checked
package 'manipulateWidget' successfully unpacked and MD5 sums checked
package 'rgl' successfully 

Warning message:
"package 'Rpdb' was built under R version 4.0.5"
Loading required package: rgl

Warning message:
"package 'rgl' was built under R version 4.0.5"

Attaching package: 'Rpdb'


The following objects are masked from 'package:bio3d':

    is.pdb, read.pdb, write.pdb


The following objects are masked from 'package:base':

    norm, replicate, unsplit




In [48]:
# Podemos descargar el archivo PDB

bio3d::get.pdb(pdb_ids[1])

[1] "./1MTO.pdb"

In [53]:
# Podemos leer los datos del PDB
pdb1 <- bio3d::read.pdb('./1MTO.pdb')

In [65]:
install.packages("remotes")
remotes::install_github("nvelden/NGLVieweR")

package 'remotes' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\javii\AppData\Local\Temp\RtmpkNpvWm\downloaded_packages


Running `R CMD build`...



* checking for file 'C:\Users\javii\AppData\Local\Temp\RtmpkNpvWm\remotes35508ae52e5\nvelden-NGLVieweR-f5badcb/DESCRIPTION' ... OK
* preparing 'NGLVieweR':
* checking DESCRIPTION meta-information ... OK
* checking for LF line-endings in source and make files and shell scripts
* checking for empty or unneeded directories
* building 'NGLVieweR_1.3.1.tar.gz'


In [67]:
library(NGLVieweR)

In [69]:
NGLVieweR("./1MTO.pdb") %>% addRepresentation("cartoon")

HTML widgets cannot be represented in plain text (need html)